# Feature Engineering

## Table of Contents
* [Libraries](#Import-Libraries)
* [Dataset](#Import_Datasets)
* [Missing Values](#Missing-Values)
* [Rare Values](#Handling-Rare-Values)
* [Encoding](#Encoding)

### Import-Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

### Import-Datasets

In [2]:
data = pd.read_csv('train.csv')
data_org = data.copy()
target = data[['SalePrice']]
data.drop(['Id'],axis=1,inplace=True)


#data.drop('Id',axis=1,inplace=True)
test = pd.read_csv('test.csv')
test.drop('Id',axis=1,inplace=True)
test_org = test.copy(deep=True)


## Missing-Values

We have two set of features, categorical features and numerical features. Lets start with numerical features and substitute the missing values with median of that features. Again, we should make some lists using list comprehension:


- nan_features : features with missing values
- num_features : features are numerical
- cat_features : features are categorical

In [3]:
features = data.columns

nan_features = [f for f in data.columns if sum(data[f].isnull())>0]
print('These {} have missing values:\n\n {}\n\n\n'.format(len(nan_features),nan_features))

num_features = [f for f in data.columns if data[f].dtypes!='O']
print('This dataset has {} numerical features :\n\n {}\n\n\n'.format(len(num_features),num_features))

cat_features = [f for f in data.columns if data[f].dtypes=='O']
print('This datset has {} numerical features :\n\n {}\n\n\n'.format(len(cat_features),cat_features))


These 19 have missing values:

 ['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']



This dataset has 37 numerical features :

 ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']



This datset has 43 numerical features :

 ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Nei

Now lets look at the portion of missing values

In [4]:
def null_percentage(df):
    null_table=pd.DataFrame([],columns=['Feature','missing'])
    for i,f in enumerate(nan_features):
        #print('{}: contains {}% missing values.'.format(f,np.round(data[f].isnull().mean(),4)))
        null_table.loc[i,'Feature'] = f
        null_table.loc[i,'missing'] = np.round(df[f].isnull().mean(),4)
        
    null_table = null_table.sort_values('missing',ascending=False)
    null_table = null_table.reset_index(drop=True)
    return null_table

In [5]:
null_percentage(data),null_percentage(test)

(         Feature missing
 0         PoolQC  0.9952
 1    MiscFeature   0.963
 2          Alley  0.9377
 3          Fence  0.8075
 4    FireplaceQu  0.4726
 5    LotFrontage  0.1774
 6     GarageType  0.0555
 7    GarageYrBlt  0.0555
 8   GarageFinish  0.0555
 9     GarageQual  0.0555
 10    GarageCond  0.0555
 11  BsmtExposure   0.026
 12  BsmtFinType2   0.026
 13  BsmtFinType1  0.0253
 14      BsmtCond  0.0253
 15      BsmtQual  0.0253
 16    MasVnrArea  0.0055
 17    MasVnrType  0.0055
 18    Electrical  0.0007,          Feature missing
 0         PoolQC  0.9979
 1    MiscFeature   0.965
 2          Alley  0.9267
 3          Fence  0.8012
 4    FireplaceQu  0.5003
 5    LotFrontage  0.1556
 6    GarageYrBlt  0.0535
 7   GarageFinish  0.0535
 8     GarageQual  0.0535
 9     GarageCond  0.0535
 10    GarageType  0.0521
 11      BsmtCond  0.0308
 12  BsmtExposure  0.0302
 13      BsmtQual  0.0302
 14  BsmtFinType1  0.0288
 15  BsmtFinType2  0.0288
 16    MasVnrType   0.011
 17    MasVn

Since the first four features have a lot of missing values, they can be nice candidates for removing from dataset. Therefore, I add a different list to keep them for the next steps.

In [6]:
rem_candidate_features = ['PoolQC','MiscFeature','Alley','Fence']

## Handling Missing values for Numerical features

* we substitue the missing value with median.

In [7]:
def Han_missing_num(df):
    num_features = [f for f in df.columns if df[f].dtypes!='O']
    nan_features = [f for f in df.columns if sum(df[f].isnull())>0]
    for f in num_features:
        if f in nan_features:
            df[f].fillna(df[f].median(),inplace=True)

In [8]:
x=num_features.copy()
x.remove('SalePrice')

In [9]:
# Handling Missing values for train
Han_missing_num(data)
print('Checking that numerical features has any missing values:',data[num_features].isnull().sum().sum())

# Handling Missing values for train
Han_missing_num(test)
print('Checking that numerical features has any missing values:',test[x].isnull().sum().sum())


Checking that numerical features has any missing values: 0
Checking that numerical features has any missing values: 0


In [10]:
def Han_missing_cat(df):

    nan_features = [f for f in df.columns if sum(df[f].isnull())>0]
    cat_features = [f for f in data.columns if data[f].dtypes=='O']
    for f in cat_features:
        if f in nan_features:
            df[f].fillna('missing',inplace=True)

In [11]:
Han_missing_cat(data)
data[cat_features].isnull().sum().sum()
print('Checking that categorical features has any missing values:',data[cat_features].isnull().sum().sum())

Han_missing_cat(test)
test[cat_features].isnull().sum().sum()
print('Checking that categorical features has any missing values:',test[cat_features].isnull().sum().sum())


Checking that categorical features has any missing values: 0
Checking that categorical features has any missing values: 0


In [12]:
print('Is train data has any missing value?',data.isnull().sum().sum())
print('Is test data has any missing value?',test.isnull().sum().sum())

Is train data has any missing value? 0
Is test data has any missing value? 0


Untile here, we do not have any missing value in our datasets.

## Handling-Rare-Values

* As it was mentioned in EDA file, we would like to distinguish the labels that has less than 1% of data as 'rare_value'.

In [13]:
def Han_cat_rare(df):
    cat_features = [f for f in df.columns if df[f].dtypes=='O']
    for f in cat_features:
        
        temp = df.groupby(f).count()['LotArea']/df.shape[0]
        temp_list = temp[temp>0.01].index
        df[f] = np.where(df[f].isin(temp_list),df[f],'Rare_value')
        

In [14]:
Han_cat_rare(data)
Han_cat_rare(test)

## Encoding

* It is the time to convert categorical variables to numerical. We convert them based on the target value. It means that if a label has greater median it will be larger than others.

In [15]:
def Enc_cat_tar(df):
    cat_features = [f for f in df.columns if df[f].dtypes=='O']
    for f in cat_features:
        temp_list = df.groupby(f).median()['SalePrice'].sort_values().index
        order = {f:i  for i,f in enumerate(temp_list)}
        df[f]=df[f].map(order)
    

In [16]:
dff = pd.concat([data,test],axis=0)

In [17]:
Enc_cat_tar(dff)

In [18]:
dff.head(10)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,1,3,3,3,1,706.0,0.0,6,4,1.0,0.0,3,150.0,1,2,1,3,0,3,2,9,9,4,1,0,4,2,4,548.0,2.0,3,2,2,4,2003.0,1710,1,2,4,5,1,2,1,0,8450,0,65.0,0,0,60,3,196.0,2,2,0,2,15,61,5,7,2,0,0,0,0,3,208500.0,2,0,1,8,856.0,1,0,2003,2003,2008
1,1262,0,0,1,3,3,3,4,978.0,0.0,4,4,0.0,1.0,3,284.0,1,1,1,3,0,3,1,4,3,4,3,1,2,2,4,460.0,2.0,3,2,2,4,1976.0,1262,0,2,4,3,1,1,1,0,9600,2,80.0,0,0,20,3,0.0,1,2,0,5,10,0,8,6,2,0,0,0,0,3,181500.0,2,0,1,6,1262.0,1,298,1976,1976,2007
2,920,866,0,1,3,3,3,2,486.0,0.0,6,4,1.0,0.0,3,434.0,1,2,1,3,0,3,2,9,9,4,3,1,4,2,4,608.0,2.0,3,2,2,4,2001.0,1786,1,2,4,5,1,2,1,0,11250,0,68.0,1,0,60,3,162.0,2,2,0,9,15,42,5,7,2,0,0,0,0,3,223500.0,2,0,1,6,920.0,1,0,2001,2002,2008
3,961,756,0,1,3,3,4,1,216.0,0.0,4,4,1.0,0.0,2,540.0,1,2,1,3,272,3,1,3,2,4,4,1,1,1,4,642.0,3.0,3,1,2,2,1998.0,1717,0,2,3,5,1,2,1,0,9550,1,60.0,1,0,70,3,0.0,1,2,0,2,17,35,5,7,2,0,0,0,0,1,140000.0,2,0,1,7,756.0,1,0,1915,1970,2006
4,1145,1053,0,1,4,3,3,3,655.0,0.0,6,4,1.0,0.0,3,490.0,1,2,1,3,0,3,2,9,9,4,3,1,4,2,4,836.0,3.0,3,2,2,4,2000.0,2198,1,2,4,5,1,2,1,0,14260,2,84.0,1,0,60,3,350.0,2,2,0,12,21,84,5,8,2,0,0,0,0,3,250000.0,2,0,1,9,1145.0,1,192,2000,2000,2008
5,796,566,320,1,1,3,3,1,732.0,0.0,6,4,1.0,0.0,3,64.0,1,2,1,3,0,3,1,9,9,1,1,0,3,1,4,480.0,2.0,3,1,2,4,1993.0,1362,1,2,4,1,1,1,1,0,14115,0,85.0,1,0,50,3,0.0,1,0,700,10,9,30,5,5,2,0,0,0,0,3,143000.0,2,0,1,5,796.0,1,40,1993,1995,2009
6,1694,0,0,1,3,3,3,3,1369.0,0.0,6,4,1.0,0.0,4,317.0,1,2,1,3,0,3,2,9,9,4,4,1,4,2,4,636.0,2.0,3,2,2,4,2004.0,1694,0,2,4,3,1,2,1,0,10084,0,75.0,0,0,20,3,186.0,4,2,0,8,18,57,5,8,2,0,0,0,0,3,307000.0,2,0,1,7,1686.0,1,255,2004,2005,2007
7,1107,983,0,1,3,3,3,2,859.0,32.0,4,1,1.0,0.0,3,216.0,1,5,1,3,228,3,1,6,6,4,3,2,2,2,4,484.0,2.0,3,2,2,4,1973.0,2090,1,2,4,5,1,1,1,0,10382,1,69.0,1,0,60,3,240.0,4,0,350,11,13,204,6,7,2,0,0,0,0,3,200000.0,2,0,1,7,1107.0,1,235,1973,1973,2009
8,1022,752,0,1,2,3,3,1,0.0,0.0,5,4,0.0,0.0,2,952.0,1,0,1,1,205,3,1,7,2,4,3,2,1,2,2,468.0,2.0,3,1,1,2,1931.0,1774,0,2,3,1,2,1,1,0,6120,0,51.0,0,0,50,1,0.0,1,2,0,4,3,0,5,7,2,0,0,0,0,1,129900.0,2,0,1,8,952.0,1,90,1931,1950,2008
9,1077,0,0,1,2,0,3,1,851.0,0.0,6,4,1.0,0.0,2,140.0,1,0,0,3,0,3,1,4,3,4,3,2,1,1,4,205.0,1.0,3,2,3,4,1939.0,1077,0,2,4,0,2,1,1,0,7420,1,50.0,0,0,190,3,0.0,1,2,0,1,5,4,6,5,2,0,0,0,0,3,118000.0,2,0,1,5,991.0,1,0,1939,1950,2008


## Handling Temporal features

In [19]:
yr_features = [f for f in data.columns if 'Yr' in f or 'Year' in f]
yr_features

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [20]:
for f in yr_features:
    if f != 'YrSold':
        dff[f] = dff['YrSold'] - dff[f]


## Adding two features:


dff['TotalArea'] = dff['TotalBsmtSF'] + dff['1stFlrSF'] + dff['2ndFlrSF'] + dff['GrLivArea'] + dff['GarageArea']
dff['TotalBath'] = dff['FullBath'] + 0.5 * dff['HalfBath']

### Normalaizing continous variables

In [21]:
con_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea','SalePrice']

for f in con_features:
    dff[f] = np.log(dff[f])

In [22]:
dff[con_features]

,LotFrontage,LotArea,1stFlrSF,GrLivArea,SalePrice
0,4.174387,9.041922,6.752270,7.444249,12.247694
1,4.382027,9.169518,7.140453,7.140453,12.109011
2,4.219508,9.328123,6.824374,7.487734,12.317167
3,4.094345,9.164296,6.867974,7.448334,11.849398
4,4.430817,9.565214,7.043160,7.695303,12.429216
5,4.442651,9.554993,6.679599,7.216709,11.870600
6,4.317488,9.218705,7.434848,7.434848,12.634603
7,4.234107,9.247829,7.009409,7.644919,12.206073
8,3.931826,8.719317,6.929517,7.480992,11.774520
9,3.912023,8.911934,6.981935,6.981935,11.678440


In [23]:
yy = dff.columns.drop('SalePrice')
en = MinMaxScaler()
dff[yy] =en.fit_transform(dff[yy])

In [24]:
dff.drop(rem_candidate_features,axis=1,inplace=True)

In [25]:
dff.to_csv('train_test.csv',index=False)